In [ ]:
%matplotlib inline
from mocpy import MOC
from mocpy import make_wcs

from astropy.wcs.utils import skycoord_to_pixel

from matplotlib.path import Path
from matplotlib.patches import PathPatch

In [ ]:
order = 6

fits_path = 'demo-data/P-GALEXGR6-AIS-FUV.fits'
moc = MOC.from_fits(fits_path)
moc = moc.degrade_to_order(order)

In [ ]:
# Computing time is related to the number of ipixels located in borders of the MOC.
# GALEX has a lot of holes and therefore there are a lot of ipixels lying in its border.
# It's especially true for deeper orders.

# The step taking the most of time is the construction of the graph from the coordinates of the ipixels
# lying in the border of all the dissociated MOC components.

# For the purpose of the demo we reduce the order of GALEX from 8 to 6
%time boundaries_l = moc.get_boundaries()

In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def add_patch_path(ax, wcs, coords, **kw_mpl_pathpatch):
    xp, yp = skycoord_to_pixel(coords=coords, wcs=wcs)
    xp = xp.flatten()
    yp = yp.flatten()
    codes = np.ones(shape=(xp.shape[0]+1,))*Path.LINETO
    codes[0] = Path.MOVETO
    codes[-1] = Path.CLOSEPOLY

    vertices = np.vstack((xp, yp)).T.tolist()
    vertices.append(vertices[0])

    path = Path(vertices, codes)

    patch = PathPatch(path, **kw_mpl_pathpatch)
    ax.add_patch(patch)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
wcs = make_wcs(crpix=[0, 0], crval=[0, 0], cdelt=[-5, 5], ctype=['RA---AIT', 'DEC--AIT'])
fig, ax = plt.subplots(1, 1, figsize=(15,15), subplot_kw={"projection": wcs})

moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color='r')
# Draw the borders, each plotted in a different color.
cmap = get_cmap(100)
for border_coords in boundaries_l:
    add_patch_path(ax=ax, wcs=wcs, coords=border_coords,
                   fill=False, color=cmap(np.random.randint(100)))

plt.axis('equal')
plt.xlabel('ra')
plt.ylabel('dec')
plt.title('P-GALEXGR6-AIS-FUV with all its borders')
plt.grid(True)
# The lines cutting the MOC are borders passing from the extreme east to the extreme west of the projection
plt.show()